In [15]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data/final_data.csv")
data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)

In [17]:
delta=df[['u_id', 'a_id', 'score']]
delta

,u_id,a_id,score
0,0,35073,6
1,13,35073,4
2,97,35073,8
3,109,35073,7
4,271,35073,4
...,...,...,...
23838,4703,4752,7
23839,5476,4752,8
23840,5956,4752,6
23841,6213,4752,9


In [18]:
data_matrix

a_id,1,5,6,16,19,20,21,25,26,27,...,38940,38993,39026,39030,39063,39195,39198,39326,39456,39533
u_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
data_matrix.columns

Index([    1,     5,     6,    16,    19,    20,    21,    25,    26,    27,
       ...
       38940, 38993, 39026, 39030, 39063, 39195, 39198, 39326, 39456, 39533],
      dtype='int64', name='a_id', length=849)

In [20]:
user_id_to_index = {} #helper function
for idx in range(len(data_matrix.index)):
    u_id = data_matrix.index[idx]
    user_id_to_index[u_id] = idx
    
user_id_to_index[104]

32

In [ ]:
anime_id_to_index = {} #helper function
for idx in range(len(data_matrix.columns)):
    a_id = data_matrix.columns[idx]
    anime_id_to_index[a_id] = idx

# anime_id_to_index[12403]

KeyError: 12403

In [ ]:
# user_id_to_index[1892]

999

In [22]:
def baseline_model(x):

    delta = df[['u_id', 'a_id', 'score']]
    avg_rating = df['score'].mean()
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    b_u=cp.Variable(num_users)
    b_i=cp.Variable(num_anime)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx
        
    R_ui=[]
    for ui in range(len(delta)):
        u = user_id_to_index[int(delta.at[ui,'u_id'])]
        i = anime_id_to_index[int(delta.at[ui,'a_id'])]
        R_ui.append((avg_rating + b_u[u] + b_i[i])) #R_ui=avg_rating+b_u[u]+b_i[i]

    training_error=[]
    for ui in range(len(delta)):
        training_error.append((R_ui[ui]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)

    regularized_u=cp.sum_squares(b_u) #sum(b_u^2)
    regularized_i=cp.sum_squares(b_i) #sum(b_i^2)

    obj=cp.Minimize(training_error_sum+ x*(regularized_u+regularized_i))
    prob=cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)
    
    R = np.zeros((num_users, num_anime))
    for u in range(num_users):
        for i in range(num_anime):
            R[u, i] = avg_rating + b_u.value[u] + b_i.value[i]
    
    print(R[70][782])

    



In [24]:
baseline_model(1000)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


status: optimal
7.293107942494531
